### Commands

In [61]:
from firecloud import api as firecloud_api
import fc_interface

In [62]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
### Parameters
namespace = "nci-mimoun-bi-org"
workspace = "CCLF_TSCA"
paths_to_samples_info = "paths_to_batches_info.xlsx"
google_bucket_id = "fc-35446f22-ea37-483a-bd6c-5e9fc56851ff"

In [ ]:
### Delete unnecessary attributes in samples
attrs = ["annotate_variants_vcf", "annotate_variants_vcf_index", "oncotated_maf", "mutect2_vcf_index", "match_normal_oncotated_maf", "gnomad_vcf", "merged_vcfs", "match_primary_tumor_bam_file"]
for idx, row in all_samples.iterrows():
    fc_interface.delete_entity_attributes(namespace, workspace, "sample", row['entity:sample_id'], attrs)

In [3]:
### Delete method repostory
firecloud_api.delete_repository_method('tsca', 'CreatePoN', 15)

<Response [200]>

In [16]:
### Delete method config
firecloud_api.delete_workspace_config(namespace, workspace, "tsca", "Mutect2_for_Normal")

In [17]:
### Delete method repository
firecloud_api.delete_repository_method('tsca', 'Mutect2_for_Normal', 1)

<Response [200]>

In [42]:
### Delete multiple method repositories
for i in range(10, 23):
    firecloud_api.delete_repository_method('tsca', 'CreatePoN', i)

In [18]:
### Update pairs
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]
res = fc_interface.upload_pairs(namespace, workspace, clean_pairs_list)

In [48]:
### Upload metadata for given batch
fc_interface.update_batch_metadata('TSCA21', \
                                  '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                                      paths_to_samples_info, \
                                        namespace, workspace, google_bucket_id)

65 Participants in this batch
95 Samples in this batch


In [70]:
### Delete sample_set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

<Response [204]>

In [74]:
### All paths to batch info
# pd.read_excel('paths_to_batches_info.xlsx').path_to_samples_info.tolist()

In [94]:
### Create cumulative PoN
batches_info = pd.read_excel(paths_to_samples_info)
PoN = fc_interface.create_panel_of_normals(batches_info.path_to_samples_info.tolist(), -1, 'CumPoN21')
fc_interface.upload_pon(PoN, 'CumPoN21', namespace, workspace)

Creating panel of 157 normals


In [104]:
### Create PoN from normals of only one batch
tsca21_path = batches_info.path_to_samples_info.tolist()[-1]
PoN = fc_interface.create_panel_of_normals(tsca21_path, -1, 'OnlyPoN21')
# fc_interface.upload_pon(PoN, 'OnlyPoN21', namespace, workspace)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.